In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
import json
import os

from datetime import datetime
import random

from langchain_openai import ChatOpenAI, AzureChatOpenAI, AzureOpenAI
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import dotenv_values
import pandas as pd

In [ ]:
import os

os.getcwd()

In [ ]:
config = dotenv_values(".env")

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=config["OPENAI_API_KEY"],
    temperature=0.5,
    top_p=0.9,
)

In [ ]:
with open("data/bva/bva_pa_recomendations.json", "r") as f:
    data = json.load(f)

In [ ]:
list_keys = list(data.keys())

In [ ]:
list_keys[0]

In [ ]:
session_event = data.get("KV23NY5")[0]

In [ ]:
list_of_keys = list(session_event.keys())

In [ ]:
system_prompt = f"""
You have a list of recomended session to assist to a visitor to a global Veterinary/ PetCare Event. Here the list of recommended sessions {{session_event}}
each event come with the following attributte : {list_of_keys}
create a list of max 3 events per day having into consideration what session better suit to the profession of the visitor
retornar el restultado en una lista con el mismo formato
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["profession", "session_event"],
    template=system_prompt
    + """Recomend from this list of session {session_event} a shortest list for a user with profession: {profession} """,
)

In [ ]:
chain = prompt | llm
print(f"length of profile: {len(prompt.template)}")

In [ ]:
ai_msg = chain.invoke({"profession": "animal surgeon", "session_event": session_event})

In [ ]:
ai_msg.content

In [ ]:
with open("data/bva/output/streams.json", "r") as json_file:
    streams = json.load(json_file)

In [ ]:
import pandas as pd


def dictionary_to_dataframe(input_dict):
    """
    Converts a dictionary with key-value pairs into a Pandas DataFrame
    with columns named 'stream' and 'description'.

    Args:
      input_dict (dict): The input dictionary.

    Returns:
      pandas.DataFrame: A DataFrame with two columns: 'stream' (keys)
                         and 'description' (values).
    """
    df = pd.DataFrame(list(input_dict.items()), columns=["stream", "description"])
    return df

In [ ]:
df_streams = dictionary_to_dataframe(streams)

In [ ]:
df_streams.to_csv("data/bva/output/streams.csv", index=False)